
## Глава 3 Теоремы теории вероятностей

**Кандидат должен знать:**

- Закон больших чисел и усиленный закон больших чисел
- Центральная предельная теорема
- Многомерные распределения

**Материалы:**
- [Райгородский курс по тер. веру лекции 8,9,12,13](https://www.youtube.com/watch?v=n7jx2MfIeX4&list=PLthfp5exSWEr8tRK-Yf-i9aXgcFJ-O16d&index=8)
- [курс ТВ  на степике из частей 2,4](https://stepik.org/course/3089)


## Задачи с кодингом


### 1. Генерация нормальной случайной величины

**Дано:**

Пусть у нас есть случайная величина $\xi \sim U[0,1]$ . 

**Задача:**
1. На основе данной случайной величины с помощью ЦПТ осуществите моделирование  ($n = 10000$) случайной величины  $\eta \sim N(\mu, \sigma^2)$

2. Постройте график полтности распределения и сравните с теоритическим. 

3. Вычислить математического ожидания и дисперсии, сравнить их с истинными значениями.

**Примечания**

1. Для генерации реализаций $\xi$ можно восппользоваться функцией numpy.random.random()
2.  Для построения теоритического графика плотности распределения нормальной величины можно воспользоваться функцией scipy.stats.norm.ppf

### 1. Генерация нормальной случайной величины
### 3. Вычисление мат. ожидания и дисперсии

Let $X$ be a uniformly distributed random variable, and $\eta$ be a normally distributed random variable.

$\eta = \frac{1}{N_x} \sum_{i=1}^{N_x} X_i$
<br>
$\mu_{\eta} = \frac{1}{N_x} \mu \left( \sum_{i=1}^{N_x} X_i \right) = \frac{1}{N_x} \sum_{i=1}^{N} \mu(X_i) = \mu_X$
<br>

$
\sigma_\eta^2 = \frac{1}{(N_x-1)^2} \sum_{i=1}^{N_x} \sigma_{x_i}^2 = \frac{N_x \cdot \sigma_x^2}{(N_x-1)^2} \approx \left[\text{if } N_x > 100\right] \approx \frac{\sigma_x^2}{N_x}
$


<br><br><br>
$\mu_X = \mu_{\eta}$
<br>
$\sigma_x^2 = N_x \cdot \sigma_\eta^2$

Now I need random uniform values $X$ with $\mu_X$ and $\sigma_x^2$:


\begin{cases}
\mu_X = \frac{a + b}{2} \\
\sigma_x^2 = \frac{(b - a)^2}{12} 
\end{cases}

\begin{cases}
a = \mu_X - \sqrt{3 \cdot \sigma_x^2} \\
b = \mu_X + \sqrt{3 \cdot \sigma_x^2}
\end{cases}


$X = a + Z \cdot (b - a) \quad \text{where} \quad Z \sim U[0, 1]$

In [1]:
import math
from tqdm import tqdm

import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def generate_eta(mu_eta, var_eta, n_x, n_eta):    
    mu_x = mu_eta
    var_x = n_x * var_eta

    a = mu_x - np.sqrt(3 * var_x)
    b = mu_x + np.sqrt(3 * var_x)

    Z = np.random.random((n_eta, n_x))
    X = a + Z * (b - a)
    
    return X.mean(axis=1)

In [3]:
mu = -7
var = 13
sigma = np.sqrt(var)
N_x = 100
N_eta = 10 ** 4

eta = generate_eta(mu, var, N_x, N_eta)

print("###  Task 3: Real E and Var VS Estimated E and Var  ###\n")
print(f"Real mu: {mu}; Estimated mu: {eta.mean()}")
print(f"Real var: {var}; Estimated var: {eta.var()}")

###  Task 3: Real E and Var VS Estimated E and Var  ###

Real mu: -7; Estimated mu: -7.012755244686017
Real var: 13; Estimated var: 13.111385867917834


In [4]:
fig = px.histogram(eta, nbins=40, title="Distribution of eta", labels={"value": "Value"}, height=700, width=500)

In [5]:
# fig.show()

![](plots/norm_distribution.png)

### 2. График полтности распределения. Сравние с теоритическим графиком

In [6]:
fig = go.Figure()

norm_x = np.linspace(norm.ppf(0.0001, loc=mu, scale=sigma), norm.ppf(0.9999, loc=mu, scale=sigma), N_eta)
norm_y = norm.pdf(norm_x, loc=mu, scale=sigma)

fig.add_trace(go.Scatter(x=norm_x, y=norm_y, mode="lines", line=dict(color="darkblue", width=5), name="norm pdf"))

hist_y, hist_x = np.histogram(eta, bins="auto", density=True)
fig.add_trace(go.Bar(x=hist_x, y=hist_y, opacity=0.6, name="histogram", marker=dict(color="darkgreen")))

fig = fig.update_layout(
    title="Normal Distribution",
    xaxis_title="Value",
    yaxis_title="Density",
    template="plotly_dark",
    legend=dict(x=0.8, y=1.0),
    height=700,
    width=500
)

In [7]:
# fig.show()

![](plots/norm_pdf.png)

### 2. Казино 

В казино имеется две игры:

**a. Кубики:**

Игрок платит 4&#36; и бросает два кубика. Если в сумме выпало 8 и более то игрок получает приз равный полученной сумме.

**b. Разгадай код:**

Есть пятизначный код (на каждой позиции стоит цифра от 0 до 9).  Игрок платит 10&#36; и пытается угадать код:
    
    - Если игрок меньше двух позиций то ничего не получает 
    - Если игрок угадал ровно две позиции из 5-ти, то он получает 50
    - Если игроку гадал ровно три позиции из 5-ти, то выигрыш 200
    - Если игроку гадал ровно четыре позиции из 5-ти, то выигрыш 5000
    - Если игрок угадал весь код правильно то выигрыш 100000.
    После каждой попытка код меняется.

**Задача:**

Пусть $\xi ,  \eta $ - случайные величины равные выигрышу(проигрышу) игрока в раунде.
1. Посчитать $E\xi$ и $E\eta$
2. Смоделировать случайные величины $\xi$ и $\eta $
3. Построить графики сходимости среднего выигрыша(проигрыша) игрока к математическому ожиданию.
4. Одинаковая ли скорость сходимости  сл. величины к своему математическому ожиданию. Почему?
4. Какое бы $n$ - количество эеспериментов вы би взяли, для оценки мат. ожидания?




## a. Кубики
### 1) Расчет мат. ожидания

In [8]:
win_amounts = np.array([0 if i < 8 else i for i in range(0, 13)]) - 4

possible_dice_values = [i + j for i in range(1, 7) for j in range(1, 7)]
count_values = [possible_dice_values.count(i) for i in range(2, 13)]
probs = list(map(lambda x: x / 36, count_values))

# skip 0 and 1 because we can't get this sums from two dices
E_xi = sum([prob * value for prob, value in zip(probs, win_amounts[2:])])  
second_moment = sum([prob * value ** 2 for prob, value in zip(probs, win_amounts[2:])])

D_xi = second_moment - E_xi ** 2

print("Value: Probability -", ",  ".join([f"{i + 2}: {count_values[i]}/36" for i in range(11)]))
print("Probs sum:", sum(count_values) / 36)
print("Theoretical E_xi:", round(E_xi, 5))
print("Theoretical D_xi:", round(D_xi, 5))

Value: Probability - 2: 1/36,  3: 2/36,  4: 3/36,  5: 4/36,  6: 5/36,  7: 6/36,  8: 5/36,  9: 4/36,  10: 3/36,  11: 2/36,  12: 1/36
Probs sum: 1.0
Theoretical E_xi: -0.11111
Theoretical D_xi: 21.82099


### 2) Моделирование случайной величины

In [9]:
np.random.seed(48)

n_samples = 9*10**3
dice_values = np.random.randint(1, 7, 2 * n_samples)
dice_sums = dice_values[::2] + dice_values[1::2]
xi_values = win_amounts[dice_sums]

print("Simulated E_xi:", xi_values.mean())

Simulated E_xi: -0.11477777777777778


### 3. Сходимость среднего выигрыша к математическому ожиданию

In [10]:
def plot(values, mean_value):
    cumulative_means = np.cumsum(values) / np.arange(1, len(values) + 1)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=np.arange(1, len(values) + 1),
        y=cumulative_means,
        mode="lines",
        name="Среднее значение выигрыша"
    ))

    fig.add_trace(go.Scatter(
        x=[1, len(values)],
        y=[mean_value, mean_value],
        mode="lines",
        line=dict(dash="dash", color="red"),
        name="Математическое ожидание"
    ))

    fig.update_layout(
        title="Сходимость среднего выигрыша к математическому ожиданию",
        xaxis_title="Количество наблюдений",
        yaxis_title="Средний выигрыш",
        legend=dict(x=0.8, y=1.0),
        template="plotly_dark"
    )

    fig.show()

In [11]:
# plot(xi_values, E_xi)

![](plots/dice_expectation.png)

### 5) Оценка количества экспериментов

In [12]:
def estimate_n(variance_x, eps_x, x):
    return ((x * np.sqrt(variance_x)) / eps_x) ** 2

In [13]:
z = 1.96
eps = 0.09

estimate_n(D_xi, eps, z)

10349.074836153028

## b. Разгадай код
### 1)  Расчет мат. ожидания

In [14]:
p = 0.1
q = 0.9
guess_code_win_amounts = np.array([0, 0, 50, 200, 5000, 100000]) - 10
code_game_probs = [math.comb(5, i) * p ** i * q ** (5 - i) for i in range(6)]
E_eta = sum([prob * value for prob, value in zip(code_game_probs, guess_code_win_amounts)])
eta_second_moment = sum([prob * value ** 2 for prob, value in zip(code_game_probs, guess_code_win_amounts)])
D_eta = eta_second_moment - E_eta ** 2

print("Probs", list(map(lambda x: round(x, 5), code_game_probs)))
print("Probs sum", sum(code_game_probs))
print("Theoretical E_eta:", round(E_eta, 5))
print("Theoretical D_eta:", round(D_eta, 5))

Probs [0.59049, 0.32805, 0.0729, 0.0081, 0.00045, 1e-05]
Probs sum 1.0
Theoretical E_eta: -1.485
Theoretical D_eta: 111683.74478


### 2) Моделирование случайной величины

In [15]:
np.random.seed(12)
N_plays = 1 * 10 ** 6

random_values_code = np.random.randint(0, 10, (N_plays, 5))
code_guesses = np.random.randint(0, 10, (N_plays, 5))

is_guess_random_code = code_guesses == random_values_code
random_guesses = is_guess_random_code.sum(axis=1)
random_guesses_win_amounts = guess_code_win_amounts[random_guesses]
print("random E", random_guesses_win_amounts.mean())

print("real E", round(E_eta, 5))

random E -1.4936
real E -1.485


### 3. Сходимость среднего выигрыша к математическому ожиданию

In [17]:
# plot(random_guesses_win_amounts, E_eta)

![](plots/code_expectation.png)

### 5) Оценка количества экспериментов

In [18]:
estimate_n(D_eta, eps, z)

52968428.87995556

### 4) Сравнение скорости сходимости

Скорости сходимости разные, так как есть значительные различия в дисперсиях случайных величин.


In [20]:
print("D_xi:", D_xi)
print("D_eta:", D_eta)

D_xi: 21.82098765432099
D_eta: 111683.74477500003


### Случайный граф


**Условие**

Пусть G(n,p) -  случайный граф на $n$ вершинах, где $p$ - это вероятность появления ребра между любыми двумя вершинами.

Обозначим за $\xi(n,p)$ - количество треугольников (т.е. полных подграфов на трёх вершинах) в случайном графе  

**Задача 1:**

1. Найти $E\xi$
2. Реализовать моделирование случайного случайного графа и функцию нахождения количества треугольников. 
3. Рассмотрим G(3,p) - случайный граф на трёх вершинах. Реализуйте $\tau$ - случайная величина равная 1 если граф полный и 0 иначе.


**Задача 2:**

- Пусть M(k) - среднее количество треугольников после k генераций случайного графа. 
- И пусть T(k)  - среднее количество треугольников после генераций случайного графа на трёх вершинах (т.е. на каждой итерации генерируем $C_n^3$ графов на трёх вершинах).

1. Верно ли что $M(k) \rightarrow E\xi$ при $k \rightarrow \infty$ 
2. Верно ли что $T(k) \rightarrow  E\xi$ при $k \rightarrow \infty$

3. Рассмотрите два графа $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$.  Построите графики  $M(k)$, $T(k)$ и $E\xi$ (константа) для заданных графов.

**Задача 3**:

Проделайте аналогичные рассуждения для $D \xi$ т.е.
- Найдите  $D \xi$  математически.
- Посчитайте дисперсию количества треугольников, k раз реализуя случайный граф.
- Посчитайте дисперсию количества треугольников, k раз генерируя  $C_n^3$ реализаций случайной величины $\tau$ .
- оцените сходимость
- Постройте графики для графов  $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$.

## Задача 1
#### 1. Найти $E\xi$
Пусть I - случайная величина равная 1 если между конкретными тремя вершинами образовался треугольник. Тогда $\xi = \sum_{i=1}^{C_n^3} I_i$.

$E\xi = E\sum_{i=1}^{C_n^3} I_i = \sum_{i=1}^{C_n^3} E I_i = C_n^3 \cdot p^3$

#### 2. Моделирования случайного графа и реализация функции нахождения количества треугольников.

In [25]:
def generate_graph(n_graphs, n_nodes, edge_p):
    dtype = np.int8 if n_nodes < 128 else np.int16
    random_values = (np.random.random((n_graphs, n_nodes, n_nodes)) < edge_p).astype(dtype)
    adjacency_matrices = np.triu(random_values, k=1)
    return adjacency_matrices


def count_triangles(graphs):
    squared_matrix = np.matmul(graphs, graphs)
    triangles_matrix = graphs * squared_matrix
    n_triangles = np.sum(np.triu(triangles_matrix, k=1), axis=(1, 2))
    return n_triangles

In [26]:
random_graphs = generate_graph(1, 5, 1)
triangles = count_triangles(random_graphs)

print("mean triangles", np.mean(triangles))

mean triangles 10.0


#### 3. G(3,p) - случайный граф на трех вершинах.

Для одного ргафа: $E\tau = 1 \cdot p^3 + 0 \cdot (1 - p^3) = p^3$

## Задача 2

In [27]:
k = 10 ** 5
n = 20
p = 0.05

print("E_xi = E_tau =", math.comb(n, 3) * p ** 3)

E_xi = E_tau = 0.14250000000000004


#### M(k)

In [28]:
%%time
random_graphs = generate_graph(k, n, p)
n_triangles_in_each_graph = count_triangles(random_graphs)

CPU times: user 1.14 s, sys: 55.4 ms, total: 1.19 s
Wall time: 1.19 s


In [29]:
M_k = np.mean(n_triangles_in_each_graph)
print("M_k:", M_k)

M_k: 0.14311


#### T(k)

In [30]:
random_graphs = (np.random.random((k, math.comb(n, 3))) < p ** 3).astype(np.int8)
N_triangles = random_graphs.sum(axis=1)

In [31]:
T_k = np.mean(N_triangles)
print("T_k:", T_k)

T_k: 0.14227


$M(k) \rightarrow E\xi$ при $k \rightarrow \infty$ <br>
$T(k) \rightarrow E\xi$ при $k \rightarrow \infty$

Оба утверждения верны

Рассмотрим $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$. И построим графики  $M(k)$, $T(k)$ и $E\xi$


In [32]:
def get_Mk_and_Tk_and_Exi(n_graphs, n_nodes, edge_p):
    E_xi = math.comb(n_nodes, 3) * edge_p ** 3
    
    g_1 = generate_graph(n_graphs, n_nodes, edge_p)
    n_triangles_1 = count_triangles(g_1)
    
    g_2 = (np.random.random((n_graphs, math.comb(n_nodes, 3))) < edge_p ** 3).astype(np.int8)
    n_triangles_2 = g_2.sum(axis=1)
    
    M_k = np.cumsum(n_triangles_1) / np.arange(1, n_graphs + 1)
    T_k = np.cumsum(n_triangles_2) / np.arange(1, n_graphs + 1)
    
    return M_k, T_k, E_xi

def make_plot(M_k, T_k, E_xi, n_graphs, n_nodes, edge_p):
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=np.arange(1, n_graphs + 1),
        y=M_k,
        mode="lines",
        line=dict(color="blue", width=3),
        name="M(k)"
    ))
    
    fig.add_trace(go.Scatter(
        x=np.arange(1, n_graphs + 1),
        y=T_k,
        mode="lines",
        line=dict(color="red", width=3),
        name="T(k)"
    ))
    
    fig.add_trace(go.Scatter(
        x=[1, n_graphs],
        y=[E_xi, E_xi],
        mode="lines",
        line=dict(dash="dash", color="darkorange", width=3),
        name="E_xi"
    ))
    
    fig.update_layout(
        title=f"Convergence of M(k) and T(k) to E_xi for G({n_nodes}, {edge_p})",
        xaxis_title="Number of graphs",
        yaxis_title="Value",
        legend=dict(x=0.94, y=1.0),
        template="plotly_dark"
    )
    
    fig.show()
    

In [45]:
np.random.seed(43)
k = 10 ** 3
n_nodes = 10

p_1 = 0.5

M_k, T_k, E_xi = get_Mk_and_Tk_and_Exi(k, n_nodes, p_1)
print("E_xi:", E_xi)
print("M_k:", M_k.mean())
print("T_k:", T_k.mean())

E_xi: 15.0
M_k: 14.60359209012895
T_k: 14.916768929094793


In [49]:
# make_plot(M_k, T_k, E_xi, k, n_nodes, p_1)

![](plots/convergence_G_10_05.png)

In [47]:
np.random.seed(41)
p_2 = 0.05

M_k, T_k, E_xi = get_Mk_and_Tk_and_Exi(k, n_nodes, p_2)
print("E_xi:", E_xi)
print("M_k:", M_k.mean())
print("T_k:", T_k.mean())

E_xi: 0.015000000000000003
M_k: 0.012020277862335947
T_k: 0.02063560314747795


In [51]:
# make_plot(M_k, T_k, E_xi, k, n_nodes, p_2)

![](plots/convergence_G_10_005.png)

In [64]:
# T_k var
math.comb(n, 3) * (p ** 3 - p ** 6)

0.14248218750000005